In [ ]:
# %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


###Essential

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_squared_error , mean_absolute_error , mean_absolute_percentage_error

In [ ]:
df = pd.read_csv('insurance.csv')
df.head(5)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
categorical_features = ['sex', 'smoker', 'region']
encode = OneHotEncoder(drop='first', sparse_output=False)
df_encoded_columns = encode.fit_transform(df[categorical_features])
encoded_df = pd.DataFrame(df_encoded_columns, columns=encode.get_feature_names_out(categorical_features))
enhanced_df = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)
# This will replace any inf or -inf values in your DataFrame with NaN, allowing you to proceed with visualizations and model training without encountering the warning.
enhanced_df.replace([np.inf, -np.inf], np.nan, inplace=True)
enhanced_df.head(4)

,age,bmi,children,charges,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0.0,1.0,0.0,0.0,1.0
1,18,33.770,1,1725.55230,1.0,0.0,0.0,1.0,0.0
2,28,33.000,3,4449.46200,1.0,0.0,0.0,1.0,0.0
3,33,22.705,0,21984.47061,1.0,0.0,1.0,0.0,0.0


### **Data Splitting and Target Transformation for Effective Training**

In this step, we **split our dataset** effectively into different sets for training and evaluation:
- **X_train**, **X_cv**, **X_test**: Features for training, cross-validation, and testing, respectively.  
- **y_train**, **y_cv**, **y_test**: Target values for training, cross-validation, and testing, respectively.

We also **apply log1p transformation** to the target variable because the range of values is very high and the distribution is not Gaussian. This transformation helps to normalize the data, making it more suitable for modeling.


In [ ]:
New_target = np.log1p(enhanced_df['charges'])

In [ ]:
# Preparing the input (X) and target (y) variables
X = enhanced_df.drop('charges', axis=1)  # Drop 'charges' column from the dataset to get features
y = New_target  # Assign the target variable

# Converting to numpy arrays
X = np.array(X)
y = np.array(y)

# Splitting the data into Training Set, Cross-Validation Set, and Test Set
X_train, x_, y_train, y_ = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% training, 40% for further split
X_cv, X_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.5, random_state=42)  # Split the remaining 40% into 50% for CV and 50% for test

# Deleting temporary variables
del x_, y_

# Displaying the shapes of the datasets and some sample values
print("Shape of X: ", X.shape)
print("Shape of y: ", y.shape)
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_cv: ", X_cv.shape)
print("Shape of y_cv: ", y_cv.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test: ", y_test.shape)

# Displaying the first 5 samples from each set
print("\nFirst 5 samples of X_train:")
print(X_train[:5])
print("First 5 values of y_train:")
print(y_train[:5])

print("\nFirst 5 samples of X_cv:")
print(X_cv[:5])
print("First 5 values of y_cv:")
print(y_cv[:5])

print("\nFirst 5 samples of X_test:")
print(X_test[:5])
print("First 5 values of y_test:")
print(y_test[:5])

Shape of X:  (1338, 8)
Shape of y:  (1338,)
Shape of X_train:  (802, 8)
Shape of y_train:  (802,)
Shape of X_cv:  (268, 8)
Shape of y_cv:  (268,)
Shape of X_test:  (268, 8)
Shape of y_test:  (268,)

First 5 samples of X_train:
[[59.   27.72  3.    0.    0.    0.    1.    0.  ]
 [60.   25.74  0.    1.    0.    0.    1.    0.  ]
 [28.   34.77  0.    0.    0.    1.    0.    0.  ]
 [19.   28.4   1.    0.    0.    0.    0.    1.  ]
 [45.   27.5   3.    1.    0.    0.    0.    1.  ]]
First 5 values of y_train:
[9.54696501 9.4045558  8.17693203 7.75470408 9.06141104]

First 5 samples of X_cv:
[[18.   31.92  0.    0.    0.    0.    0.    0.  ]
 [27.   29.15  0.    1.    1.    0.    1.    0.  ]
 [26.   23.7   2.    1.    0.    0.    0.    1.  ]
 [53.   36.86  3.    0.    1.    1.    0.    0.  ]
 [54.   47.41  0.    0.    1.    0.    1.    0.  ]]
First 5 values of y_cv:
[ 7.69938071  9.81178312  8.1563183  10.75069489 11.06306053]

First 5 samples of X_test:
[[43.    26.7    2.     0.     1.    

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def evaluate_regression_performance(true_values, predicted_values):
    """
    This function evaluates the performance of a regression model using various metrics.
    It prints out detailed information about:
    1. Mean Squared Error (MSE)
    2. Scaled Mean Squared Error
    3. Mean Absolute Error (MAE)
    4. Root Mean Squared Error (RMSE)
    5. Mean Absolute Percentage Error (MAPE)
    """

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(true_values, predicted_values)
    print(f"--- Performance Evaluation ---")
    print(f"\n1. Mean Squared Error (MSE): {mse:.4f}")
    print("   - MSE calculates the average of the squared differences between the predicted and actual values. "
          "It penalizes larger errors more heavily, making it a sensitive measure of overall prediction accuracy.\n")

    # Scaled Mean Squared Error (MSE/2)
    mse_scaled = mse / 2
    print(f"2. Scaled Mean Squared Error (MSE/2): {mse_scaled:.4f}")
    print("   - This is a scaled version of MSE, used for specific use cases. Here, it is scaled down by dividing by 2 "
          "to see a more normalized value.\n")

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(true_values, predicted_values)
    print(f"3. Mean Absolute Error (MAE): {mae:.4f}")
    print("   - MAE represents the average of the absolute errors. It is more intuitive than MSE since it measures the "
          "average magnitude of errors without considering their direction (positive or negative).\n")

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print(f"4. Root Mean Squared Error (RMSE): {rmse:.4f}")
    print("   - RMSE gives an idea of the magnitude of the error in the same units as the target variable. "
          "It is useful when you want to assess the absolute error and is particularly sensitive to large errors.\n")

    # Calculate Mean Absolute Percentage Error (MAPE)
    mape = mean_absolute_percentage_error(true_values, predicted_values)
    print(f"5. Mean Absolute Percentage Error (MAPE): {mape:.4f}%")
    print("   - MAPE expresses the error as a percentage of the actual values, providing a relative measure of prediction accuracy. "
          "It is useful when comparing performance across different datasets or models.\n")


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.5 MB/s eta 0:00:00


###**Essential Libraries for XGBoost** 🚀  

To efficiently work with **XGBoost**, make sure you have these key libraries installed:  


In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
def objective(trial):
    # Define hyperparameters to optimize
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'verbosity': 1,  # Fixed
        'random_state': 42  # Fixed
    }

    # Train model
    model = XGBRegressor(**params)
    model.fit(X_train, y_train)

    # Predict and evaluate
    preds = model.predict(X_cv)
    mse = mean_squared_error(y_cv, preds)
    return mse  # Minimize MSE


In [ ]:
import optuna
study = optuna.create_study(direction='minimize')  # Minimize error
study.optimize(objective, n_trials=100)

[I 2025-02-06 17:20:30,994] A new study created in memory with name: no-name-8af5f70c-f93e-4a1c-a956-73b36758aa89
[I 2025-02-06 17:20:31,132] Trial 0 finished with value: 0.1395435439131087 and parameters: {'n_estimators': 129, 'learning_rate': 0.07071378116644443}. Best is trial 0 with value: 0.1395435439131087.
[I 2025-02-06 17:20:31,732] Trial 1 finished with value: 0.1711019431892842 and parameters: {'n_estimators': 972, 'learning_rate': 0.06666825333134373}. Best is trial 0 with value: 0.1395435439131087.
[I 2025-02-06 17:20:31,921] Trial 2 finished with value: 0.1550701598621269 and parameters: {'n_estimators': 299, 'learning_rate': 0.006197386077833728}. Best is trial 0 with value: 0.1395435439131087.
[I 2025-02-06 17:20:32,355] Trial 3 finished with value: 0.17294618216744131 and parameters: {'n_estimators': 782, 'learning_rate': 0.05192577105649636}. Best is trial 0 with value: 0.1395435439131087.
[I 2025-02-06 17:20:32,944] Trial 4 finished with value: 0.15041278283044726 and

### **XGBoost Hyperparameter Tuning Results**
"We see that **Best Hyperparameters Found: Random Forest**, but previously our model was **Random Forest**, so we forgot to change the name. I am not going to re-run the notebook as it may cause a data mismatch, so this is just for reference.  

Final information:  

**Best Hyperparameters Found: XGBoost**  
- Parameters: `{'n_estimators': 257, 'learning_rate': 0.022247240955584915}`  
- Best Objective Value (MSE): **0.12782969267871547**  

**Best Trials Summary:**  
- Trials: `[FrozenTrial(number=99, state=1, values=[0.12782969267871547], datetime_start=datetime.datetime(2025, 2, 6, 17, 20, 59, 423803)...]`

In [ ]:
print("Best Hyperparameters Found: Random Forest ")
print(f"  Parameters: {study.best_params}")
print(f"  Best Objective Value /mse: {study.best_value}")
print("Best Trials Summary:")
print(f"  Trials: {study.best_trials}")

Best Hyperparameters Found: Random Forest 
  Parameters: {'n_estimators': 257, 'learning_rate': 0.022247240955584915}
  Best Objective Value /mse: 0.12782969267871547
Best Trials Summary:
  Trials: [FrozenTrial(number=99, state=1, values=[0.12782969267871547], datetime_start=datetime.datetime(2025, 2, 6, 17, 20, 59, 423803), datetime_complete=datetime.datetime(2025, 2, 6, 17, 20, 59, 636208), params={'n_estimators': 257, 'learning_rate': 0.022247240955584915}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.001, step=None)}, trial_id=99, value=None)]


In [ ]:
best_param = study.best_params
XGmodel = XGBRegressor(**best_param)
XGmodel.fit(X_train , y_train)
y_train_predict = XGmodel.predict(X_train)
print('For training set {Make no sense but }')
evaluate_regression_performance(y_train , y_train_predict)

For training set {Make no sense but }
--- Performance Evaluation ---

1. Mean Squared Error (MSE): 0.0473
   - MSE calculates the average of the squared differences between the predicted and actual values. It penalizes larger errors more heavily, making it a sensitive measure of overall prediction accuracy.

2. Scaled Mean Squared Error (MSE/2): 0.0236
   - This is a scaled version of MSE, used for specific use cases. Here, it is scaled down by dividing by 2 to see a more normalized value.

3. Mean Absolute Error (MAE): 0.1137
   - MAE represents the average of the absolute errors. It is more intuitive than MSE since it measures the average magnitude of errors without considering their direction (positive or negative).

4. Root Mean Squared Error (RMSE): 0.2174
   - RMSE gives an idea of the magnitude of the error in the same units as the target variable. It is useful when you want to assess the absolute error and is particularly sensitive to large errors.

5. Mean Absolute Percentage 

In [ ]:
y_cv_predict = XGmodel.predict(X_cv)
print('For CV set')
evaluate_regression_performance(y_cv , y_cv_predict)

For CV set
--- Performance Evaluation ---

1. Mean Squared Error (MSE): 0.1278
   - MSE calculates the average of the squared differences between the predicted and actual values. It penalizes larger errors more heavily, making it a sensitive measure of overall prediction accuracy.

2. Scaled Mean Squared Error (MSE/2): 0.0639
   - This is a scaled version of MSE, used for specific use cases. Here, it is scaled down by dividing by 2 to see a more normalized value.

3. Mean Absolute Error (MAE): 0.2022
   - MAE represents the average of the absolute errors. It is more intuitive than MSE since it measures the average magnitude of errors without considering their direction (positive or negative).

4. Root Mean Squared Error (RMSE): 0.3575
   - RMSE gives an idea of the magnitude of the error in the same units as the target variable. It is useful when you want to assess the absolute error and is particularly sensitive to large errors.

5. Mean Absolute Percentage Error (MAPE): 0.0228%
   - 

In [ ]:
y_test_predict = XGmodel.predict(X_test)
print('For Test set')
evaluate_regression_performance(y_test , y_test_predict)

For Test set
--- Performance Evaluation ---

1. Mean Squared Error (MSE): 0.1533
   - MSE calculates the average of the squared differences between the predicted and actual values. It penalizes larger errors more heavily, making it a sensitive measure of overall prediction accuracy.

2. Scaled Mean Squared Error (MSE/2): 0.0767
   - This is a scaled version of MSE, used for specific use cases. Here, it is scaled down by dividing by 2 to see a more normalized value.

3. Mean Absolute Error (MAE): 0.1987
   - MAE represents the average of the absolute errors. It is more intuitive than MSE since it measures the average magnitude of errors without considering their direction (positive or negative).

4. Root Mean Squared Error (RMSE): 0.3916
   - RMSE gives an idea of the magnitude of the error in the same units as the target variable. It is useful when you want to assess the absolute error and is particularly sensitive to large errors.

5. Mean Absolute Percentage Error (MAPE): 0.0218%
   

### **Final Model Selection and Deployment**  

After evaluating various models, we achieved the **lowest error** using the **XGBoost Regressor**. Given its superior performance, we **saved the trained model**, making it ready for deployment.  

With this, our **machine learning model is now complete and optimized** for accurate predictions. 🚀  


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/Insurance project"


'3)  insurance_Decision_tree.ipynb'    insurance_parametric_regression.ipynb
 insurance.csv			      'XGBoost 4)insurance_Decision_tree'
 insurance_ipynb_visualization.ipynb


In [ ]:
drive_path = "/content/drive/My Drive/Colab Notebooks/Insurance project/best_xgboost_model.json"

try:
    XGmodel.save_model(drive_path)
    print(f"Model saved successfully in Google Drive at: {drive_path}")
except Exception as e:
    print(f"Error saving model: {e}")# Saves in JSON format

Model saved successfully in Google Drive at: /content/drive/My Drive/Colab Notebooks/Insurance project/best_xgboost_model.json
